In [ ]:
!pip install gpytorch

In [ ]:
training_iterations = 500
lr=0.1
num=10

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import gpytorch
from gpytorch import kernels, means, models, mlls, settings, constraints
from gpytorch import distributions as distr
import json
import scipy
import scipy.stats
import tqdm

DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Use the simplest form of GP model, exact inference
class GPModel(models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)

        self.ln_e=nn.Embedding(3, 2, padding_idx=None)
        self.depth_e=nn.Embedding(13, 2, padding_idx=None)
        self.head_e=nn.Embedding(4, 2, padding_idx=0)
        self.mlp_e=nn.Embedding(4, 2, padding_idx=0)
        
        base_kernel = kernels.MaternKernel(nu=1.5)
        
        self.mean_module = means.MultitaskMean(
            means.ConstantMean(), num_tasks=len(train_y[0])
        )
        self.covar_module = kernels.MultitaskKernel(
            kernels.ArcKernel(base_kernel), num_tasks=len(train_y[0]), rank=1
        )
        
        self.encoder_layer1 = nn.TransformerEncoderLayer(d_model=4, nhead=2,dim_feedforward=4,dropout=0.5,activation="relu",batch_first=True,layer_norm_eps=1e-5,norm_first=True)
        self.encoder_layer2 = nn.TransformerEncoderLayer(d_model=4, nhead=2,dim_feedforward=4,dropout=0.5,activation="relu",batch_first=True,layer_norm_eps=1e-5,norm_first=True)
        
        self.linear = nn.Linear(300,8)
        
    def forward(self, x):
        t1=self.depth_e(torch.tensor([12]).long().to(DEVICE))
        t2=self.ln_e(x[:,0].unsqueeze(1))
        
        t1=torch.stack([t1 for _ in range(len(t2))])
        
        e=torch.cat((t1,t2),axis=-1)

        for i in range(1,25):
            t1=self.depth_e(torch.tensor(int(i/2)).long().to(DEVICE)).reshape((1,-1))
            t1=torch.stack([t1 for _ in range(len(t2))])
            if((i%2)==0):
                t2=self.mlp_e(x[:,i].unsqueeze(1))
                tt=torch.cat((t1,t2),axis=-1)
                e=torch.cat((e,tt),axis=1)
            else:
                t2=self.head_e(x[:,i].unsqueeze(1))
                tt=torch.cat((t1,t2),axis=-1)
                e=torch.cat((e,tt),axis=1)
        x=e
        
        xl=[x]
        
        x=self.encoder_layer1(x)
        xl.append(x)
        x=self.encoder_layer2(x)
        xl.append(x)
        
        x=torch.cat(xl,axis=-1)
        
        x=x.squeeze()
        
        x=torch.flatten(x,1,-1)
        
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return distr.MultitaskMultivariateNormal(mean_x, covar_x)

In [ ]:
with open('search_res.json', 'r') as f:
    search_new_res = json.load(f)

In [ ]:
cp_l=search_new_res['cp_l']
score_l=search_new_res['score_l']
score_l=np.asarray(score_l)
score_l.shape
inconsideration=np.argsort(-score_l,axis=0)[:num,:]
task_listL=[[] for _ in range(8)]
for i in range(8):
    for j in range(num):
        task_listL[i].append(cp_l[inconsideration[j][i]])
task_listL

In [ ]:
with open('CVPR_2022_NAS_Track2_train.json', 'r') as f:
    train_data = json.load(f)
print(train_data['arch1'])
print('train_num:',len(train_data.keys()))
def convert_X(arch_str):
    temp_arch = []
    for elm in arch_str:
        if elm == 'l':
            temp_arch.append(0)
        elif elm == 'j':
            temp_arch.append(1)
        elif elm == 'k':
            temp_arch.append(2)
        else: 
            temp_arch.append(int(elm))
    return(temp_arch)
train_list = [[],[],[],[],[],[],[],[]]
arch_list_train = []
name_list = ['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank', 'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']
for key in train_data.keys():
    for idx, name in enumerate(name_list):
        train_list[idx].append(train_data[key][name])#8*500
    arch_list_train.append(convert_X(train_data[key]['arch']))#500
print(len(train_list))

In [ ]:
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    x = (x - np.mean(x)) / np.std(x)
    return x

tmp=[0]
for i in range(1,37):
    if(i%3==0):
        continue
    tmp.append(i)

In [ ]:
with open('CVPR_2022_NAS_Track2_test.json', 'r') as f:
    test_data = json.load(f)
arch_list = []
for key in test_data.keys():
    arch_list.append(convert_X(test_data[key]['arch']))
test_x  = np.array(arch_list)[:,tmp]
test_x = torch.tensor(test_x).long().to(DEVICE)
print(test_x.shape)

In [ ]:
task_list=[1,2,3]
X_all_k, Y_all_k  = np.array(arch_list_train)[:,tmp], ZscoreNormalization(np.array(train_list).T)
train_X, train_Y = torch.tensor(X_all_k).long().to(DEVICE),torch.tensor(Y_all_k).float().to(DEVICE)
print(train_X.shape,train_Y.shape)

In [ ]:
res_l=[]
for tsk in range(8):
    total_res=torch.zeros(([99500]),dtype=torch.float32)
    for task_list_id in range(len(task_listL[tsk])):
        task_list=task_listL[tsk][task_list_id]
        print("task:",tsk,"; id:",task_list_id,"; cp:",task_list,end=" ")
        train_x=train_X.clone()
        train_y=train_Y[:,task_list].clone()
        
        likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=len(train_y[0])).to(DEVICE)
        model = GPModel(train_x, train_y, likelihood).to(DEVICE)

        model.train()
        likelihood.train()

        optimizer = torch.optim.Adam(model.parameters(),lr)

        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        print("train ",end="")
        for i in range(training_iterations):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            
        model.eval()
        likelihood.eval()
        single_res=None
        print("test ",end='')
        with torch.no_grad():
            for i in range(0,int(99500/995)):
                inp=test_x[i*995:(i+1)*995]
                observed_pred = likelihood(model(inp)).mean
                if(i==0):
                    single_res=observed_pred
                else:
                    single_res=torch.cat((single_res,observed_pred),axis=0)
        single_res=single_res[:,task_list.index(tsk)]
        
        total_res=total_res+single_res
        print("!")
    res_l.append(total_res)

In [ ]:
final_res=torch.stack(res_l,-1)
print(final_res.shape)
rank=torch.argsort(torch.argsort(final_res,axis=0),axis=0)

In [ ]:
for idx,key in enumerate(test_data.keys()):
    test_data[key]['cplfw_rank'] = int(rank[idx][0])
    test_data[key]['market1501_rank'] = int(rank[idx][1])
    test_data[key]['dukemtmc_rank'] = int(rank[idx][2])
    test_data[key]['msmt17_rank'] = int(rank[idx][3])
    test_data[key]['veri_rank'] = int(rank[idx][4])
    test_data[key]['vehicleid_rank'] = int(rank[idx][5])
    test_data[key]['veriwild_rank'] = int(rank[idx][6])
    test_data[key]['sop_rank'] = int(rank[idx][7])

In [ ]:
print('Ready to save results!')
with open('taskEnsemble_'+str(num)+'task_'+str(training_iterations)+'epoch_'+str(lr)+'lr'+'_submit.json', 'w') as f:
    json.dump(test_data, f)